In [25]:
import json
import os
import copy
import random 
import numpy as np
from client import *
import matplotlib.pyplot as plt

In [34]:
Team_Key = "VPN75t7UGO44XsrQlo9KuKkg8yRrqgSfyh13GITsQnyvLLyvV5" 
Team_Name = "Platypus_Perry"
max_limit = 10 
min_limit = -10
generations = 10
array_len = 11
population_size = 8
chromosome_size = 11
Mutation_Probability = 0.2         # probabilty that it will mutate
Mutation_Difference_Scale = 1000   # adding noice on mutation to actual value ratio
initial_array = [0.0, -1.45799022e-12, -2.28980078e-13,  4.62010753e-11, -1.75214813e-10, -1.83669770e-15,  8.52944060e-16,  2.29423303e-05, -2.04721003e-06, -1.59792834e-08,  9.98214034e-10]
valid_err_best_weights = 0
train_err_best_weights = 0
best_weights_set = []
fitness_best_weights = 0

In [31]:
def add_noise(arr):
    for id, val in np.ndenumerate(arr):
        if np.random.random() < Mutation_Probability:
            range_var = val/Mutation_Difference_Scale
            arr[id] = arr[id] + np.random.uniform(-range_var,range_var)
    return np.clip(arr, Min_Len, Max_Len)

In [33]:
def populate(arr):
    arra = []
    for i in range(1, population_size):
        arra.append(arr[i])
    arra = np.array(arra, dtype=np.double)
    arra = add_noise(arra)
population = populate(initial_array)

In [2]:
def Roulette_Wheel_Selection(arr ,fitness_sum):
    random_nu = random.uniform(0,fitness_sum)
    for id, val in np.ndenumerate(arr):
        if (val > random_nu):
            return id
    return len(arr) - 1 

In [36]:
def get_gen(i):
    j = i
    selected_population = []
    crossovered_population = []
    mutated_population = []
    
    #finding errors
    fitness = []
    train_error = []
    valid_error = []
    for chromosomes in population:
        train_err, valid_err = get_errors(Team_Key, list(chromosomes))
        train_error.append(train_err)
        valid_error.append(valid_err)
        fitness.append(-(train_err + valid_err))

    #updating err values
    best_fit_id = np.argmax(fitness)
    if ( (fitness_best_weights != 0) and (fitness[best_fit_id] > fitness_best_weights)):
        fitness_best_weights = fitness[best_fit_id]
        valid_err_best_weights = valid_error[best_fit_id]
        train_err_best_weights = train_error[best_fit_id]
        best_weights_set[:] = population[best_fit_id]
    
    scaled_fitness = (fitness - np.min(fitness)) / np.ptp(fitness) #fitness is scaled from 0 to 1
    partial_sum = []
    sum_fitness = np.sum(scaled_fitness)
    part = 0.0
    for val in scaled_fitness:
        part = part + val
        partial_sum.append(part)

    crossover =[]
    for i in range(0, population_size//2):
        selected_f = population[Roulette_Wheel_Selection(partial_sum ,sum_fitness)]# selection is left
        selected_s = population[Roulette_Wheel_Selection(partial_sum ,sum_fitness)]
        selected_population.append(selected_f)
        selected_population.append(selected_s)

        # crossover 
        crossovered_f = copy(selected_f)
        crossovered_s = copy(selected_s)
        crossover_point = np.random.randint(chromosome_size)
        crossovered_f[0:crossover_point] = selected_f[0:crossover_point]
        crossovered_s[0:crossover_point] = selected_s[0:crossover_point]
        crossovered_population.append(crossovered_f)
        crossovered_population.append(crossovered_s)
        crossover.append(crossovered_f)
        crossover.append(crossovered_s)
    
    #mutation
    crossover = np.array(crossover, dtype=np.double)
    population = add_noise(crossover)
    mutated_population = population

    #creating generation files
    file_name = "generations/" + "generations_" + str(j) + ".txt"
    selected_population = np.array(selected_population)
    crossovered_population = np.array(crossovered_population)
    mutated_population = np.array(mutated_population)
    with open(file_name, 'w') as write_file:
        json.dump(selected_population.tolist(), write_file)
        write_file.write('\n' + '\n')
        json.dump(crossovered_population.tolist(), write_file)
        write_file.write('\n' + '\n')
        json.dump(mutated_population.tolist(), write_file)


In [ ]:
if (os.path.isdir("generations") != 1):
    os.mkdir("generations")
for i in range(0,generations):
    get_gen(i)